In [21]:
%pip install langchain
## Import LangChain related modules
#coding=utf-8
import openai
import os
import json

os.environ['OPENAI_API_KEY'] = '*'

os.environ["http_proxy"]="127.0.0.1:7890"
os.environ["https_proxy"]="127.0.0.1:7890"
from langchain import PromptTemplate
import re
import tenacity
from typing import List, Dict, Callable
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import RegexParser
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    BaseMessage,
)

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simpleNote: you may need to restart the kernel to use updated packages.



In [22]:
## Define classes
class DialogueAgent:
    def __init__(
        self,
        name: str,
        system_message: SystemMessage,
        model: ChatOpenAI,
    ) -> None:
        self.name = name
        self.system_message = system_message
        self.model = model
        self.prefix = f"{self.name}: "
        self.reset()
        
    def reset(self):
        self.message_history = ["Here is the conversation so far."]

    def send(self) -> str:
        """
        Applies the chatmodel to the message history
        and returns the message string
        """
        message = self.model(
            [
                self.system_message,
                HumanMessage(content="\n".join(self.message_history + [self.prefix])),
            ]
        )
        return message.content

    def receive(self, name: str, message: str) -> None:
        """
        Concatenates {message} spoken by {name} into message history
        """
        self.message_history.append(f"{name}: {message}")


class DialogueSimulator:
    def __init__(
        self,
        agents: List[DialogueAgent],
        selection_function: Callable[[int, List[DialogueAgent]], int],
    ) -> None:
        self.agents = agents
        self._step = 0
        self.select_next_speaker = selection_function
        
    def reset(self):
        for agent in self.agents:
            agent.reset()

    def inject(self, name: str, message: str):
        """
        Initiates the conversation with a {message} from {name}
        """
        for agent in self.agents:
            agent.receive(name, message)

        # increment time
        self._step += 1

    def step(self) -> tuple[str, str]:
        # 1. choose the next speaker
        speaker_idx = self.select_next_speaker(self._step, self.agents)
        speaker = self.agents[speaker_idx]

        # 2. next speaker sends message
        message = speaker.send()

        # 3. everyone receives message
        for receiver in self.agents:
            receiver.receive(speaker.name, message)

        # 4. increment time
        self._step += 1

        return speaker.name, message
class BiddingDialogueAgent(DialogueAgent):
    def __init__(
        self,
        name,
        system_message: SystemMessage,
        bidding_template: PromptTemplate,
        model: ChatOpenAI,
    ) -> None:
        super().__init__(name, system_message, model)
        self.bidding_template = bidding_template
        
    def bid(self) -> str:
        """
        Asks the chat model to output a bid to speak
        """
        prompt = PromptTemplate(
            input_variables=['message_history', 'recent_message'],
            template = self.bidding_template
        ).format(
            message_history='\n'.join(self.message_history),
            recent_message=self.message_history[-1])
        bid_string = self.model([SystemMessage(content=prompt)]).content
        return bid_string
        

In [23]:
# characters = []
case_description = """On September 17, 2012, Henan Henghe Real Estate Co., Ltd. ("Henghe Company") and Zhongtian Construction Group Co., Ltd. ("Zhongtian Company") signed a Contract for Construction of the Construction Project of the Henghe International Business Convention and Exhibition Center, under which Zhongtian Company would construct the project involved in the case. On June 25, 2013, Henghe Company issued a Bid Winning Notice to Zhongtian Company, notifying Zhongtian Company that it succeeded in the bidding for the Henghe International Business Convention and Exhibition Center Project on Kaiyuan Avenue, Luolong District, Luoyang City. On June 26, 2013, Henghe Company and Zhongtian Company signed a Contract for Construction of Construction Project, in which the both parties agreed on the construction term, project cost, liability for breach of contract, and other matters related to the project. After the contract was signed, Zhongtian Company entered the site for construction. During the construction, as Henghe Company defaulted on project cost payment, Zhongtian Company delivered letters to Henghe Company on November 12, 2013, November 26, 2013, and December 23, 2014, claiming that Henghe Company should immediately pay project cost in arrears, and should pay liquidated damages and be liable for the corresponding loss as agreed in the contract. In April and May 2014, Henghe Company signed a Construction Project Cost Consultation Contract with TAHP Project Management (Beijing) Co., Ltd. ("TAHP Company") and retained TAHP Company to conduct a settlement examination of the project involved in the case. On November 3, 2014, TAHP Company issued a Report on the Settlement Examination of Henghe International Business Exhibition Center. Henghe Company, Zhongtian Company, and TAHP Company affixed their official seals to, and signed for confirmation, the examination summary schedule in the examination report. On November 24, 2014, Zhongtian Company received a notice that the Intermediate People's Court of Jiaozuo City, Henan Province would have the project involved in the case auctioned as per the application by another creditor of Henghe Company. On December 1, 2014, the Ninth Construction Company of Zhongtian Company submitted to the Intermediate People's Court of Jiaozuo City, Henan Province a Letter Regarding the Auction of the Henghe International Business Convention and Exhibition Center Project under Construction, stating that Zhongtian Company, as the contractor of the Henghe International Business Convention and Exhibition Center Project under construction, had completed construction works worth 287 million yuan since the commencement of construction of the project, and requested the confirmation of its priority of payment and participation in the auction throughout the process. Zhongtian Company and Henghe Company both admitted that the project involved in the case were suspended on February 5, 2015."""
def get_agent(case_description):
    description_systemMessage = [SystemMessage(content = f"""
    Here is the case description of a dispute over a contract for construction of a construction project: {case_description}\
    Extract only the parties who need to participate in the [court hearing] to solve the dispute.'''\
    Answer template (Valid list format!!!):
    {["name of party1", "name of party2", "..."]}
    """)]
    description_response = ChatOpenAI(temperature=0)(description_systemMessage).content
    return description_response
description_response = get_agent(case_description)
characters = eval(description_response)
print(characters)

['Henan Henghe Real Estate Co., Ltd.', 'Zhongtian Construction Group Co., Ltd.', 'TAHP Project Management (Beijing) Co., Ltd.']


In [24]:
for character in characters:
    print(character)

Henan Henghe Real Estate Co., Ltd.
Zhongtian Construction Group Co., Ltd.
TAHP Project Management (Beijing) Co., Ltd.


In [25]:
word_limit = 50

In [26]:
descriptor_system_message = SystemMessage(
     content="You can add detail to the each parties claim and goal.")
def generate_claims(character):
    generate_claims_prompt = [
        descriptor_system_message,
        HumanMessage(content = f"""
    Here is the case description of a dispute over a contract for construction of a construction project: {case_description}\
    You are the {character} in this case.\
    You want to solve the dispute on the court hearing\
    Please reply with your claim or goal you want to achieve in the court.\
    Make your claim or goal clear and specific.\
    Reply with a third person tongue.\
    Reply within {word_limit}.
    Do not add anything else.
    """)]
    claims = ChatOpenAI(temperature=0.3)(generate_claims_prompt).content
    return claims

In [27]:
def generate_character_header(character,character_claim):
    return f"""{case_description}
Your are {character} in this dispute.\
You are in an arbitrational hearing.\
This is the goal you want to reach through the hearing: {character_claim}
"""

def generate_character_system_message(character,character_header):
    return SystemMessage(content=(
    f"""{character_header}
Speak in the first person from the perspective of {character}.
For describing your own body movements, wrap your description in '*'.
Do not change roles!
Do not speak from the perspective of anyone else.
Speak only from the perspective of {character}.
Stop speaking the moment you finish speaking from your perspective.
Never forget to keep your response to {word_limit} words!
Do not add anything else.
    """
    ))
character_claims = [generate_claims(character) for character in characters]
character_headers = [generate_character_header(character, character_claim) for character, character_claim in zip(characters, character_claims)]
character_system_messages = [generate_character_system_message(character, character_headers) for character, character_headers in zip(characters, character_headers)]


In [28]:
for character, character_claim, character_header, character_system_message in zip(characters, character_claims, character_headers, character_system_messages):
    print(f'\n\n{character} Claim:')
    print(f'\n{character_claim}')
    print(f'\n{character_header}')
    print(f'\n{character_system_message.content}')



Henan Henghe Real Estate Co., Ltd. Claim:

As Henan Henghe Real Estate Co., Ltd., our goal in the court hearing is to dispute the claims made by Zhongtian Construction Group Co., Ltd. regarding project cost payment and liquidated damages. We aim to prove that we have fulfilled our obligations under the Contract for Construction of Construction Project and that Zhongtian Company's claims are unfounded. Additionally, we seek to protect our rights as the owner of the Henghe International Business Convention and Exhibition Center Project and ensure that the project is not auctioned off without proper consideration of all parties involved.

On September 17, 2012, Henan Henghe Real Estate Co., Ltd. ("Henghe Company") and Zhongtian Construction Group Co., Ltd. ("Zhongtian Company") signed a Contract for Construction of the Construction Project of the Henghe International Business Convention and Exhibition Center, under which Zhongtian Company would construct the project involved in the case

In [29]:
# game_description = f"""Here is the topic for the presidential debate: {topic}.
# The presidential candidates are: {', '.join(character_names)}."""

# player_descriptor_system_message = SystemMessage(
#     content="You can add detail to the description of each presidential candidate.")

# def generate_character_description(character_name):
#     character_specifier_prompt = [
#         player_descriptor_system_message,
#         HumanMessage(content=
#             f"""{game_description}
#             Please reply with a creative description of the presidential candidate, {character_name}, in {word_limit} words or less, that emphasizes their personalities. 
#             Speak directly to {character_name}.
#             Do not add anything else."""
#             )
#     ]
#     character_description = ChatOpenAI(temperature=1.0)(character_specifier_prompt).content
#     return character_description

# def generate_character_header(character_name, character_description):
#     return f"""{game_description}
# Your name is {character_name}.
# You are a presidential candidate.
# Your description is as follows: {character_description}
# You are debating the topic: {topic}.
# Your goal is to be as creative as possible and make the voters think you are the best candidate.
# """

# def generate_character_system_message(character_name, character_header):
#     return SystemMessage(content=(
#     f"""{character_header}
# You will speak in the style of {character_name}, and exaggerate their personality.
# You will come up with creative ideas related to {topic}.
# Do not say the same things over and over again.
# Speak in the first person from the perspective of {character_name}
# For describing your own body movements, wrap your description in '*'.
# Do not change roles!
# Do not speak from the perspective of anyone else.
# Speak only from the perspective of {character_name}.
# Stop speaking the moment you finish speaking from your perspective.
# Never forget to keep your response to {word_limit} words!
# Do not add anything else.
#     """
#     ))

# character_descriptions = [generate_character_description(character_name) for character_name in character_names]
# character_headers = [generate_character_header(character_name, character_description) for character_name, character_description in zip(character_names, character_descriptions)]
# character_system_messages = [generate_character_system_message(character_name, character_headers) for character_name, character_headers in zip(character_names, character_headers)]
                                                                                                                                       

## Output parser for bids

In [30]:
class BidOutputParser(RegexParser):
    def get_format_instructions(self) -> str:
        return 'Your response should be an integer delimited by angled brackets, like this: <int>.'     
    
bid_parser = BidOutputParser(
    regex=r'<(\d+)>', 
    output_keys=['bid'],
    default_output_key='bid')

## Generate bidding system message

In [31]:
def generate_character_bidding_template(character_header):
    bidding_template = (
    f"""{character_header}

```
{{message_history}}
```

Output a number on the scale of 1 to 10, where 1 is not contradictory and 10 is extremely contradictory, rate how contradictory the following message is to your ideas.

```
{{recent_message}}
```

{bid_parser.get_format_instructions()}
Do nothing else.
    """)
    return bidding_template

character_bidding_templates = [generate_character_bidding_template(character_header) for character_header in character_headers]
                                      

In [32]:
for character, bidding_template in zip(characters, character_bidding_templates):
    print(f'{character} Bidding Template:')
    print(bidding_template)

Henan Henghe Real Estate Co., Ltd. Bidding Template:
On September 17, 2012, Henan Henghe Real Estate Co., Ltd. ("Henghe Company") and Zhongtian Construction Group Co., Ltd. ("Zhongtian Company") signed a Contract for Construction of the Construction Project of the Henghe International Business Convention and Exhibition Center, under which Zhongtian Company would construct the project involved in the case. On June 25, 2013, Henghe Company issued a Bid Winning Notice to Zhongtian Company, notifying Zhongtian Company that it succeeded in the bidding for the Henghe International Business Convention and Exhibition Center Project on Kaiyuan Avenue, Luolong District, Luoyang City. On June 26, 2013, Henghe Company and Zhongtian Company signed a Contract for Construction of Construction Project, in which the both parties agreed on the construction term, project cost, liability for breach of contract, and other matters related to the project. After the contract was signed, Zhongtian Company ente

## Use an LLM to create an elaborate on debate topic

In [33]:
topic_specifier_prompt = [
    SystemMessage(content="You can specify the goal of the hearing more clearly.."),
    HumanMessage(content=
        f"""{case_description}
        
        You are the arbitrator in the hearing to solve the dispute.\
        Please make an opeing of the arbitration hearing.\
        You should speak in the style of an arbitrator!\
        Lead the parties in the hearing to present their claims.\
        Please reply with the specified topic in {word_limit} words or less. 
        Speak directly to the parties: {*characters,}.
        Do not add anything else."""
        )
]
specified_topic = ChatOpenAI(temperature=1.0)(topic_specifier_prompt).content

print(f"Detailed topic:\n{specified_topic}\n")

Detailed topic:
Dear parties, welcome to the arbitration hearing. Today, our goal is to resolve the dispute regarding project cost payments and liquidated damages related to the Henghe International Business Convention and Exhibition Center construction project. I invite each party to present their claims and arguments clearly and concisely. Henan Henghe Real Estate Co., Ltd., please present your arguments regarding the project cost and payment default. Zhongtian Construction Group Co., Ltd., please present your claims regarding the completed construction works and priority of payment. TAHP Project Management (Beijing) Co., Ltd., please provide any relevant information regarding the settlement examination of the project.



## Define the speaker selection function

In [36]:
@tenacity.retry(stop=tenacity.stop_after_attempt(2),
                    wait=tenacity.wait_none(),  # No waiting time between retries
                    retry=tenacity.retry_if_exception_type(ValueError),
                    before_sleep=lambda retry_state: print(f"ValueError occurred: {retry_state.outcome.exception()}, retrying..."),
                    retry_error_callback=lambda retry_state: 0) # Default value when all retries are exhausted
def ask_for_bid(agent) -> str:
    """
    Ask for agent bid and parses the bid into the correct format.
    """
    bid_string = agent.bid()
    bid = int(bid_parser.parse(bid_string)['bid'])
    return bid
# for character in characters:
#     bid_strings = ask_for_bid(character)
#     print(bid_strings)

In [37]:
import numpy as np

def select_next_speaker(step: int, agents: List[DialogueAgent]) -> int:
    bids = []
    for agent in agents:
        bid = ask_for_bid(agent)
        bids.append(bid)
    # if len(bids) == 0:
    #     raise ValueError("No agent with bids found.")
    # randomly select among multiple agents with the same bid
    max_value = np.max(bids)

    max_indices = np.where(bids == max_value)[0]
    idx = np.random.choice(max_indices)
    
    print('Bids:')
    for i, (bid, agent) in enumerate(zip(bids, agents)):
        print(f'\t{agent.name} bid: {bid}')
        if i == idx:
            selected_name = agent.name
    print(f'Selected: {selected_name}')
    print('\n')
    return idx

## Main loop

In [38]:
characters = []
for character, character_system_message, bidding_template in zip(characters, character_system_messages, character_bidding_templates):
    characters.append(BiddingDialogueAgent(
        name=character,
        system_message=character_system_message,
        model=ChatOpenAI(temperature=0),
        bidding_template=bidding_template,
    ))

In [39]:
max_iters = 10
n = 0

simulator = DialogueSimulator(
    agents=characters,
    selection_function=select_next_speaker
)
simulator.reset()
simulator.inject('Debate Moderator', specified_topic)
print(f"(Debate Moderator): {specified_topic}")
print('\n')

while n < max_iters:
    name, message = simulator.step()
    print(f"({name}): {message}")
    print('\n')
    n += 1

(Debate Moderator): Dear parties, welcome to the arbitration hearing. Today, our goal is to resolve the dispute regarding project cost payments and liquidated damages related to the Henghe International Business Convention and Exhibition Center construction project. I invite each party to present their claims and arguments clearly and concisely. Henan Henghe Real Estate Co., Ltd., please present your arguments regarding the project cost and payment default. Zhongtian Construction Group Co., Ltd., please present your claims regarding the completed construction works and priority of payment. TAHP Project Management (Beijing) Co., Ltd., please provide any relevant information regarding the settlement examination of the project.




ValueError: zero-size array to reduction operation maximum which has no identity